In [5]:
import pickle
import pandas as pd
import numpy as np
from collections import Counter

from matplotlib import pyplot as plt
%matplotlib inline

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
def simulateData(n = 1000, max_sentences = 50, max_tokens = 500, max_sentlen = 10):
    xCorpus = Counter()
    qid = list(range(n))
    qdf = pd.DataFrame(qid, columns=['query_id'])
    qdf['n_sentences'] = np.random.randint(low=1, high=(max_sentences + 1), size=(n,))
    sdf = pd.DataFrame(qid, columns=['query_id'])
    #sdf.set_index('query_id', inplace=True)
    sdf['tstokens'] = ''

    for i in range(qdf['n_sentences'].max()):
        sdf['stokens_%i' % i] = ''

    for q_i, nsent_i in zip(qdf['query_id'], qdf['n_sentences']):
        for sent_j in range(nsent_i):
            sentlen_i = np.random.randint(low=1, high=(nsent_i + 1), size=(1,))[0]
            rand_sent = np.random.randint(low=1, high=(max_tokens + 1), size=(sentlen_i,)).tolist()
            for r_l in rand_sent:
                xCorpus[r_l] += 1
                
            sdf.loc[q_i, 'stokens_%i' % sent_j] = ' '.join([str(j) for j in rand_sent])

        # Randomly choosing size of summary, which one of the indices to select and then subsetting them
        summary_size_i = np.random.randint(low=1, high=(nsent_i + 1), size=(1,))[0]
        selected_i = sorted(list(set(np.random.randint(low=0, high=(nsent_i), size=(summary_size_i,)))))
        truesummary_i = sdf.loc[q_i,:][['stokens_%i' %i for i in selected_i]]

        sdf.loc[q_i, 'tstokens'] = ' '.join(truesummary_i)
        
    return sdf, xCorpus

In [6]:
inputfile = "/home/francisco/GitHub/DQN-Event-Summarization/data/cnn_tokenized/cnn_data_corpus.csv"
inputdict = "/home/francisco/GitHub/DQN-Event-Summarization/data/cnn_tokenized/cnn_total_corpus_smry.csv"

qdf = pd.read_csv(inputfile)
qdict = pd.read_csv(inputdict)
corpus_dict = dict(zip(qdict['id'], qdict['token']))

#qdf, corpus_dict = simulateData()

/home/francisco/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
qdf.head()

,query_id,qtokens,tstokens,stokens_0,stokens_1,stokens_2,stokens_3,stokens_4,stokens_5,stokens_6,...,stokens_115,stokens_116,stokens_117,stokens_118,stokens_119,stokens_120,stokens_121,stokens_122,stokens_123,stokens_124
0,0,20001 102 33 149 105 48,0 1 2 3 4 5 6 7 20001 8 9 10 0 11 12 13 14 0 2...,0 1 2 3 4 5 6 7,20001 8 9 10 0 11 12 13 14 0 20001 15 16 17 18...,0 23 51 14 52 53 54 55 56 23 57 58 59 9 60 61 ...,68 69 70 71 72 0 73 74 75 76 77 15 78 79 80 81...,0 94 90 37 69 14 0 95 9 26 78 79 96 20001 97 9...,37 96 107 108 15 109 4 20001 110 101 48 111 99...,123 0 124 125 126 127 128 129 4 130 131 37 132...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,20001 102 319 550 551 19316 549 553 566,78 79 549 411 550 44 551 552 549 84 553 20001 ...,78 79 549 411 550 44 551 552 549 84 553 20001 ...,37 561 102 562 3 563 564 565 8 37 20001 23 566...,37 558 102 551 84 570 571 572 23 573 74 337 26...,394 84 570 574 56 105 558 20001 74 575 271 15 ...,84 582 583 82 83 551 582 583 584 110 585 3 586...,37 595 20001 32 37 553 596 161 202 302 302 315...,112 574 56 3 603 604 605 606 72 607 298 32 196...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,690 691 3 19317 591 20001 37 1510,37 549 788 41 20001 756 8 78 79 572 808 809 81...,37 549 788 41 20001 756 8 78 79 572,37 789 110 37 579 790 791 105 792 153 793 794 ...,808 809 810 811 627 812 32 78 79 572 6 8 253 8...,8 822 68 110 339 823 824 37 825 826 827 110 78...,8 822 37 831 832 833 826 827 238 808 809 810 8...,834 78 79 572 835 37 35 836 837 110 253 838 3 ...,843 46 126 46 269 271 110 78 79 572 105 624 13...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,690 1289 19318 747 15 886 1170 1171 99 20001 837,78 79 549 280 1168 105 1169 37 78 79 572 538 8...,78 79 549 280 1168 105 1169 37 78 79 572 538 8...,37 1171 538 1179 71 280 284 9 1 1180 99 1172 9...,1185 1187 37 1188 1189 627 248 20001 1190 318 ...,1194 1195 507 9 551 1196 1188 1197 430 8 37 11...,153 105 9 720 720 1208 889 9 837 889 507 1209 ...,248 6 14 37 78 79 572 591 67 35 1211,37 551 1188 1212 1213 3 1184 15 37 799 800 801...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,19319 1398 46 1385 5599 1430 8 14494 20001,78 79 549 139 1380 1381 318 9 1382 1383 8 1384...,78 79 549 139 1380 1381 318 9 1382 1383 8 1384...,9 1390 1391 869 8 1185 1392 1393 609 1394 223 ...,852 8 37 1397 316 1398,37 574 56 869 8 578 3 579 580 1399 15 90 7 140...,37 869 23 1405 896 37 1406 1407 110 1408 1409 ...,455 99 37 1414 1415 316 1036 15 1416 1333 14 1...,37 869 123 1421 280 1034 1422 852 218 15 37 80...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
queries = qdf['query_id']
sentences = qdf[[x for x in qdf.columns if 'stokens_' in x]]

true_summaries = {}
for queryid, true_summary in zip(queries, qdf['tstokens']):
    true_summaries[queryid] = Counter([int(x) for x in true_summary.split(" ")])

In [9]:
def make_bow_vector(sentence, word_to_ix):
    # tf BOW model
    vec = torch.zeros(len(word_to_ix))
    for token in sentence:
        vec[token] += 1
    
    return vec.view(1, -1)

In [10]:
# Make a global variable
xs = torch.zeros(sentences.shape[0], len(corpus_dict))
xs.shape

torch.Size([83566, 111212])

In [ ]:
# from joblib import Parallel, delayed
# torch.save(tst, 'tst.pkl')
# torch.load('tst.pkl')
# Parallel(n_jobs=-1)(delayed(tensorSentences)(i) for i in range(sentences.shape[0]))

In [11]:
tokens = sentences['stokens_0'][0].split(" ")
test = make_bow_vector([int(s) for s in tokens], corpus_dict)
test

tensor([[1., 1., 1.,  ..., 0., 0., 0.]])

In [12]:
train_xs = torch.zeros(sentences.shape[1], len(corpus_dict))
# train_ys = torch.from_numpy(np.random.random((n, 2))).float()  # Simulating the output

# bow for each sentence
#for i, row in enumerate(sentences['stokens_0']):
idx = 0
tmp_sentence = sentences.ix[idx,:]
sentences_i = tmp_sentence[tmp_sentence.notnull()]
for j, sentence_j in enumerate(sentences_i):
    tokens = sentence_j.split(" ")
    if len(tokens) > 0:
        train_xs[j, :] = make_bow_vector([int(s) for s in tokens], corpus_dict)
    else:
        pass

In [13]:
train_xs

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [2., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

torch_file = '/home/francisco/GitHub/DQN-Event-Summarization/data/cnn_tokenized/sentence_tensor.pt'
truesummary_file = '/home/francisco/GitHub/DQN-Event-Summarization/data/cnn_tokenized/true_summaries_corpus.pkl'
pickle.dump(true_summaries, open(truesummary_file, 'wb'))

torch.save(train_xs, torch_file)

In [17]:
def rougueScores(genSummary, refSummary):
    genTotal, refTotal, intersection = 0, 0, 0
    for token in list(set(list(refSummary.keys()) + list(genSummary.keys()) )):
        intersection += min(refSummary[token], genSummary[token])
        refTotal += refSummary[token]
        genTotal += genSummary[token]

    recall = intersection / float(refTotal) if float(refTotal) > 0 else 0
    prec   = intersection / float(genTotal) if float(genTotal) > 0 else 0
    f1 = (2 * recall * prec) / (recall + prec) if (recall + prec) > 0 else 0
    
    return recall, prec, f1

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

class BoWRegressor(nn.Module):  # inheriting from nn.Module!
    # calls the init function of nn.Module.  Dont get confused by syntax, always do it in an nn.Module
    def __init__(self, outputsize, nsentences, vocab_size, nlayers, nunits):
        super(BoWRegressor, self).__init__()        
        self.nlayers = nlayers
        self.inputlayer = nn.Linear(vocab_size, nunits)
        self.hiddenlayer = nn.Linear(nunits, nunits)
        self.outputlayer = nn.Linear(nunits, outputsize)
        
        
    def forward(self, bowmatrix):
        hiddenlayer = self.inputlayer(bowmatrix)
        for _ in range(self.nlayers):
            hiddenlayer = self.hiddenlayer(hiddenlayer)

        return F.relu(self.outputlayer(hiddenlayer))

# Specifying model parameters

In [46]:
SKIP = 0
SELECT = 1

nsent = train_xs.shape[0]
vsize = train_xs.shape[1]
outsize = 2
nhlayers = 1
nhunits = 128
rand_rate = 1.0

model = BoWRegressor(outsize, nsent, vsize, nhlayers, nhunits)
action = torch.from_numpy(np.zeros(shape=(nsent, 2))).int()

In [71]:
rouge_preds = model(autograd.Variable(train_xs))
qMax, qIndx = rouge_preds.max(dim=1)

if np.random.uniform() < rand_rate and rand_rate > 0:
    # Randomly choosing either 0 or 1 some percent of the time
    qIndx = np.random.randint(0, 2, qIndx.shape[0])

# Choosing action
for i, q_j in enumerate(qIndx):
    action[i, :] = 0 
    action[i, q_j] = 1
    
predQonActions = torch.masked_select(rouge_preds, autograd.Variable(action.byte()))

# TO DO

In [ ]:
for i in range(nsent):
    if action[i, SELECT] == 1:
        train_xs

In [ ]:
qdf.ix[0, :]

In [81]:
sentences_i[0].split(' ')

['0', '1', '2', '3', '4', '5', '6', '7']

In [75]:
train_xs[0, :]

tensor([1., 1., 1.,  ..., 0., 0., 0.])

In [74]:
action

tensor([[0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        

In [41]:
recall, prec, f1 = rougeScores(ts_tokenized, Counter(curr_summary.split(" ")))


In [18]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

lossf = {'loss': [], 'epoch': []}
for epoch in range(100):
    model.zero_grad()
    rougue_preds = model(autograd.Variable(train_xs))
    loss = loss_function(rougue_preds, autograd.Variable(train_ys))
    loss.backward()
    optimizer.step()
    lossf['loss'].append(loss.data[0])
    lossf['epoch'].append(epoch)
    
perf = pd.DataFrame(lossf)

perf.plot(y='loss', figsize=(12, 6))
plt.show()

In [26]:
preds.shape

torch.Size([125, 2])

In [ ]:
train_xs[]

what do  I need to do
1. need to save sentences to tensors
2. need to save counter summaries to pickle objects
3. need to make code run on small examples

In [23]:
print([int(s) for s in sentences['stokens_0'][0].split(" ") ])
print([corpus_dict[int(w)] for w in sentences['stokens_0'][0].split(" ") if int(w) in corpus_dict])

[0, 1, 2, 3, 4, 5, 6, 7]
['i', 'm', '45', 'and', 'my', 'son', 'is']


In [ ]:
trueSummary = Counter([1, 1, 1, 2, 2, 2, 2, 3, 3])
predSummary0 = Counter([1, 1, 1, 2, 2, 2, 2, 3, 3])
predSummary1 = Counter([1, 1, 1, 2, 2, 2, 2, 3, 3, 4])
predSummary2 = Counter([1, 1, 1, 2, 2, 2, 2, 3])
predSummary3 = Counter([4])

In [ ]:
print(rougueScores(predSummary0, trueSummary))
print(rougueScores(predSummary1, trueSummary))
print(rougueScores(predSummary2, trueSummary))
print(rougueScores(predSummary3, trueSummary))

In [ ]:
outputsize = 2 
vocab_size = len(corpus_dict)
model = BoWRegressor(outputsize, vocab_size)

In [ ]:
model.zero_grad()

loss_function = nn.MSELoss()

rougue_preds = model(autograd.Variable(train_xs))
loss = loss_function(rougue_preds, autograd.Variable(train_ys))

print(
    (( (rougue_preds.data).cpu().numpy() - (autograd.Variable(train_ys).data).cpu().numpy() ) **2 ).mean()
     )
print(loss)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

lossf = {'loss': [], 'epoch': []}
for epoch in range(100):
    model.zero_grad()
    
    rougue_preds = model(autograd.Variable(train_xs))
    loss = loss_function(rougue_preds, autograd.Variable(train_ys))
    loss.backward()
    optimizer.step()
    lossf['loss'].append(loss.data[0])
    lossf['epoch'].append(epoch)
    
perf = pd.DataFrame(lossf)

perf.plot(y='loss', figsize=(12, 6))
plt.show()

In [ ]:
true_summaries[0]